# GPT-4o-mini As Jugde

In [1]:
PROMPT = {}

In [32]:
PROMPT['evaluation_gen_questions'] = '''
### **Hướng Dẫn Đánh Giá**

**Vai trò:**
Bạn là một chuyên gia được giao nhiệm vụ đánh giá hai câu trả lời cho cùng một câu hỏi dựa trên ba tiêu chí: **Độ toàn diện, Sự đa dạng**, và **Độ hữu ích**.

**Mục tiêu:**
Bạn sẽ đánh giá hai câu trả lời cho cùng một câu hỏi dựa trên ba tiêu chí:

- **Độ toàn diện:** Câu trả lời chi tiết đến mức nào trong việc bao quát tất cả các khía cạnh và chi tiết của câu hỏi?
- **Sự đa dạng:** Câu trả lời có phong phú và mang lại nhiều góc nhìn, quan điểm khác nhau về câu hỏi không?
- **Độ hữu ích:** Câu trả lời có giúp người đọc hiểu rõ vấn đề và đưa ra các đánh giá, quyết định sáng suốt không?

**Yêu cầu:**
Với mỗi tiêu chí, hãy chọn câu trả lời tốt hơn (Câu trả lời 1 hoặc Câu trả lời 2) và giải thích lý do tại sao. Sau đó, chọn ra câu trả lời tổng thể tốt hơn dựa trên ba tiêu chí này.

---

### **Đầu Vào Đánh Giá**

Dưới đây là câu hỏi: <<q>>

Dưới đây là hai câu trả lời:
**Câu trả lời 1:**
<<a1>>
(Kết thúc câu trả lời 1)

**Câu trả lời 2:**
<<a2>>
(Kết thúc câu trả lời 2)

Hãy đánh giá cả hai câu trả lời dựa trên ba tiêu chí đã nêu và cung cấp giải thích chi tiết cho từng tiêu chí.

Xuất kết quả đánh giá của bạn theo định dạng JSON như sau:

```json
{
  "Độ toàn diện": {
    "Người thắng": "[Câu trả lời 1 hoặc Câu trả lời 2]",
    "Giải thích": "[Cung cấp giải thích tại đây]"
  },
  "Sự đa dạng": {
    "Người thắng": "[Câu trả lời 1 hoặc Câu trả lời 2]",
    "Giải thích": "[Cung cấp giải thích tại đây]"
  },
  "Độ hữu ích": {
    "Người thắng": "[Câu trả lời 1 hoặc Câu trả lời 2]",
    "Giải thích": "[Cung cấp giải thích tại đây]"
  },
  "Người thắng tổng thể": {
    "Người thắng": "[Câu trả lời 1 hoặc Câu trả lời 2]",
    "Giải thích": "[Tóm tắt tại sao đây là câu trả lời tốt nhất dựa trên ba tiêu chí trên]"
  }
}
```
'''

## Thiết lập LLM
- GPT-4o-mini
- 8k input context

In [ ]:
!pip install openai

In [6]:
!pip install python-dotenv

In [7]:
from dotenv import load_dotenv
import os
import openai

# Load file .env
load_dotenv()

# Lấy API key từ .env
api_key = os.getenv("OPENAI_API_KEY")

In [8]:
# Gán API key cho OpenAI
openai.api_key = api_key

In [15]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
)

response.choices[0].message.content

'This is a test. How can I assist you further?'

In [143]:
def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2, # this is the degree of randomness of the model's output
    )
    print(model)
    return response.choices[0].message.content

In [126]:
def get_valid_json_gpt40mini(prompt,max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        print(f"attempt : {attempts}")
        HEAD = """In your previous response you responded with the wrong request format below, now you have to do it again and respond with the correct JSON as described below"""

        if attempts != 0:
            prompt = HEAD + "\n" + prompt

        res_second = get_completion(prompt)  # Gọi LLM
        try:
            dlist = json.loads(res_second.strip("```").strip("json").strip("\n```"))
            print("Định dạng JSON hợp lệ!")
            return dlist
        except json.JSONDecodeError:
            attempts += 1
            print(f"JSON không hợp lệ, thử lại ({attempts}/{max_attempts})...")
    return "Not"

# Check 125 câu hỏi được gen ra

In [17]:
import json
with open('/content/LIGHTRAG_HYBRID_QA.json', 'r', encoding='utf-8') as f:
    LIGHTRAG_HYBRID_QA = json.load(f)

In [20]:
with open('/content/LIGHTRAG_NAIVE_QA.json', 'r', encoding='utf-8') as f:
    LIGHTRAG_NAIVE_QA = json.load(f)

In [22]:
len(LIGHTRAG_HYBRID_QA)

125

In [23]:
len(LIGHTRAG_NAIVE_QA)

125

In [24]:
LIGHTRAG_HYBRID_QA[0]

{'q': 'Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?',
 'a': 'Theo quy định trong Luật Hôn nhân và Gia đình 2014, có một số điều kiện cần thiết để thực hiện việc kết hôn mà các bên cần lưu ý nhằm tránh vi phạm pháp luật. Dưới đây là những điểm chính:\n\n### 1. Đối tượng kết hôn\nNguyên tắc đầu tiên là cả hai bên phải là cá nhân đủ tuổi kết hôn, theo đó, nam phải từ đủ 20 tuổi và nữ phải từ đủ 18 tuổi. Đây là điều kiện cơ bản và quan trọng nhất, nếu không đáp ứng yêu cầu về độ tuổi, việc kết hôn sẽ không hợp pháp.\n\n### 2. Tự nguyện\nKết hôn phải được thực hiện trên cơ sở sự tự nguyện của cả hai bên. Bất kỳ hình thức ép buộc nào sẽ dẫn đến việc doanh từ mọi hiệu lực pháp lý. Các bên cần đảm bảo rằng không có sự uy hiếp hoặc ép buộc từ phía gia đình hoặc xã hội trong quyết định kết hôn.\n\n### 3. Tình trạng hôn nhân\nMột bên không được phép kết hôn trong khi vẫn đang có vợ hoặc chồng. Điều này được quy định tại Đ

In [25]:
LIGHTRAG_NAIVE_QA[0]

{'q': 'Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?',
 'a': '### Các Điều Kiện Kết Hôn Theo Quy Định Của Pháp Luật Việt Nam\n\nKhi tư vấn về các điều kiện kết hôn, rất quan trọng để khách hàng hiểu rõ các quy định của Luật Hôn nhân và Gia đình, nhằm tránh các vấn đề pháp lý liên quan đến việc kết hôn trái pháp luật. Dưới đây là một số điểm cần lưu ý:\n\n#### 1. Điều Kiện Độ Tuổi\n\nLuật quy định rõ ràng về độ tuổi kết hôn. Cụ thể:\n\n- **Nam**: Từ đủ **20 tuổi** trở lên.\n- **Nữ**: Từ đủ **18 tuổi** trở lên.\n\nKhách hàng cần lưu ý rằng việc kết hôn trước độ tuổi quy định là vi phạm pháp luật và có thể bị xử lý.\n\n#### 2. Tự Nguyện Trong Quyết Định Kết Hôn\n\nMột trong những nguyên tắc cơ bản là:\n\n- **Sự tự nguyện**: Việc kết hôn phải xuất phát từ quyết định tự nguyện của cả hai bên, không bị ép buộc, cưỡng chế hay lừa dối.\n\nKhách hàng cần đảm bảo rằng cả hai bên đều đồng ý và không chịu bất kỳ áp lực nào.

In [106]:
RESULTS_GEN_QUESTIONS = []

In [107]:
RESULTS_GEN_QUESTIONS

[]

In [78]:
from tqdm import tqdm
import random

In [108]:
VER4 = []

In [109]:
VER4

[]

In [110]:
for i in tqdm(range(len(LIGHTRAG_HYBRID_QA))):
  hybrid = LIGHTRAG_HYBRID_QA[i]
  naive = LIGHTRAG_NAIVE_QA[i]

  print(LIGHTRAG_HYBRID_QA[i]['q'])
  print(LIGHTRAG_NAIVE_QA[i]['q'])

  prompt = PROMPT['evaluation_gen_questions']

  prompt = prompt.replace('<<q>>', LIGHTRAG_HYBRID_QA[i]['q'])

  number = random.randint(1, 45)
  print(number)
  if number % 2 == 0:
    prompt = prompt.replace('<<a1>>', LIGHTRAG_NAIVE_QA[i]['a']) # hy # na
    prompt = prompt.replace('<<a2>>', LIGHTRAG_HYBRID_QA[i]['a']) #na # hy
  else:
    prompt = prompt.replace('<<a1>>', LIGHTRAG_HYBRID_QA[i]['a']) # hy # na
    prompt = prompt.replace('<<a2>>', LIGHTRAG_NAIVE_QA[i]['a'])

  res = get_valid_json_gpt40mini(prompt)

  if res == "Not":
    continue

  a= {}
  a['q'] = LIGHTRAG_HYBRID_QA[i]['q']
  a['res'] = res
  a['num'] = number

  print(a['res'])

  VER4.append(a)

  #print(prompt)


  0%|          | 0/125 [00:00<?, ?it/s]

Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?
Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?
39
attempt : 0


  1%|          | 1/125 [00:08<16:50,  8.15s/it]

Định dạng JSON hợp lệ!
{'Độ toàn diện': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn chi tiết và đầy đủ về các điều kiện kết hôn theo quy định của pháp luật, bao gồm độ tuổi, tự nguyện, tình trạng hôn nhân, quan hệ huyết thống, quy trình đăng ký và giấy tờ cần thiết. Trong khi đó, Câu trả lời 2 thiếu một số khía cạnh như quan hệ huyết thống và giấy tờ cần thiết.'}, 'Sự đa dạng': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 không chỉ đề cập đến các điều kiện kết hôn mà còn mở rộng ra các trường hợp cấm kết hôn và năng lực hành vi dân sự, mang lại nhiều góc nhìn hơn về vấn đề này. Câu trả lời 1 chủ yếu tập trung vào các điều kiện cơ bản mà không đề cập đến các trường hợp cấm kết hôn.'}, 'Độ hữu ích': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp thông tin rõ ràng và cụ thể về các điều kiện cần thiết để kết hôn, giúp người đọc dễ dàng hiểu và áp dụng. Câu trả lời 2 cũng hữu ích nhưng không chi tiết bằng Câu t

  2%|▏         | 2/125 [00:15<16:14,  7.92s/it]

...

Định dạng JSON hợp lệ!
{'Độ toàn diện': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn chi tiết và đầy đủ về các nguyên tắc pháp lý liên quan đến phân chia tài sản trong quá trình ly hôn. Nó không chỉ nêu rõ các loại tài sản mà còn giải thích các nguyên tắc công bằng, thỏa thuận, thời điểm chia tài sản và yêu cầu tố tụng. Trong khi đó, Câu trả lời 2 mặc dù cũng đề cập đến nhiều khía cạnh nhưng không chi tiết bằng Câu trả lời 1.'}, 'Sự đa dạng': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 thể hiện sự đa dạng hơn trong việc trình bày các nguyên tắc pháp lý, bao gồm cả yếu tố lỗi trong phân chia tài sản, điều này không được đề cập trong Câu trả lời 1. Nó cũng nhấn mạnh đến quyền lợi và nghĩa vụ tài sản, điều này giúp mở rộng góc nhìn về vấn đề phân chia tài sản.'}, 'Độ hữu ích': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp thông tin rõ ràng và có cấu trúc tốt, giúp người đọc dễ dàng hiểu và áp dụng các nguyên

 99%|█████████▉| 124/125 [12:56<00:06,  6.05s/it]

Định dạng JSON hợp lệ!
{'Độ toàn diện': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn chi tiết về các quy định trong Luật Hôn nhân và Gia đình năm 2014, bao gồm các khía cạnh như bảo vệ lợi ích chính đáng, nghĩa vụ cấp dưỡng và quyền thăm nom. Nó cũng nêu rõ các phương pháp tư vấn cho phụ huynh, giúp họ hiểu rõ hơn về quyền lợi của trẻ em trong tình huống ly hôn.'}, 'Sự đa dạng': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 mang lại nhiều góc nhìn khác nhau về việc bảo vệ quyền lợi của trẻ em, đặc biệt là việc tôn trọng ý kiến của trẻ và tạo ra môi trường sống ổn định. Nó cũng nhấn mạnh tầm quan trọng của việc tham khảo ý kiến chuyên gia pháp lý và đạt được thỏa thuận giữa cha mẹ, điều này tạo ra sự đa dạng trong các phương pháp tư vấn.'}, 'Độ hữu ích': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 không chỉ cung cấp thông tin chi tiết về quy định pháp luật mà còn đưa ra các bước cụ thể mà phụ huynh có thể thực hiện

100%|██████████| 125/125 [13:02<00:00,  6.26s/it]

Định dạng JSON hợp lệ!
{'Độ toàn diện': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn tổng quát và chi tiết về các phương án tư vấn, bao gồm hòa giải, tư vấn tâm lý và tư vấn pháp luật. Nó không chỉ nêu rõ các phương pháp mà còn giải thích cách thức hoạt động của từng phương pháp, giúp người đọc hiểu rõ hơn về cách thức xử lý mâu thuẫn gia đình.'}, 'Sự đa dạng': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 đưa ra nhiều góc nhìn khác nhau, bao gồm sự tham gia của người có uy tín và chức sắc tôn giáo, cũng như nhấn mạnh tầm quan trọng của giáo dục và truyền thông trong gia đình. Điều này tạo ra sự phong phú trong cách tiếp cận vấn đề, không chỉ dừng lại ở các phương pháp tư vấn mà còn mở rộng ra các khía cạnh văn hóa và xã hội.'}, 'Độ hữu ích': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp thông tin cụ thể và rõ ràng về các phương pháp tư vấn, giúp người đọc có thể áp dụng vào thực tế. Nó không chỉ nêu ra các

In [50]:
len(RESULTS_GEN_QUESTIONS)

125

In [51]:
RESULTS_GEN_QUESTIONS[8]

{'q': 'Các quy định pháp luật liên quan đến bảo vệ quyền lợi của trẻ em trong các vụ ly hôn và bạo lực gia đình là gì?',
 'res': {'Độ toàn diện': {'Người thắng': 'Câu trả lời 1',
   'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn toàn diện về quy định pháp luật liên quan đến quyền lợi của trẻ em trong các vụ ly hôn và bạo lực gia đình. Nó không chỉ đề cập đến quyền nuôi con mà còn giải thích rõ ràng về các yếu tố mà Tòa án xem xét, cũng như trách nhiệm của cha mẹ. Trong khi đó, Câu trả lời 2 mặc dù cũng đề cập đến quyền bày tỏ ý kiến của trẻ em nhưng không đi sâu vào các khía cạnh khác như trách nhiệm của cha mẹ và các biện pháp can thiệp từ cơ quan chức năng.'},
  'Sự đa dạng': {'Người thắng': 'Câu trả lời 2',
   'Giải thích': 'Câu trả lời 2 mang lại sự đa dạng hơn khi đề cập đến quyền bày tỏ ý kiến của trẻ em và nhấn mạnh vai trò của gia đình, xã hội và nhà nước trong việc bảo vệ trẻ em. Nó cũng chỉ ra trách nhiệm của cha mẹ trong việc tạo ra môi trường an toàn cho trẻ, điều này k

In [55]:
hybrid_win_toan_dien = 0
hybrid_win_su_da_ngang = 0
hybrid_win_do_huu_ich = 0
hybrid_win_overall = 0

In [58]:
hybrid_win_toan_dien = 0
hybrid_win_su_da_dang = 0
hybrid_win_do_huu_ich = 0
hybrid_win_overall = 0

for res in RESULTS_GEN_QUESTIONS:
  print(res['q'])
  #print(res['res'])

  r = res['res']

  print('Độ toàn diện:')
  print(r['Độ toàn diện']['Người thắng'])
  win = r['Độ toàn diện']['Người thắng']
  if win == 'Câu trả lời 1':
    hybrid_win_toan_dien += 1

  print('Sự đa dạng:')
  print(r['Sự đa dạng']['Người thắng'])
  if r['Sự đa dạng']['Người thắng'] == 'Câu trả lời 1':
    hybrid_win_su_da_dang += 1

  print('Độ hữu ích:')
  print(r['Độ hữu ích']['Người thắng'])
  if r['Độ hữu ích']['Người thắng'] == 'Câu trả lời 1':
    hybrid_win_do_huu_ich += 1

  print('Kết quả cuối: ')
  print(r['Người thắng tổng thể']['Người thắng'])
  if r['Người thắng tổng thể']['Người thắng'] == 'Câu trả lời 1':
    hybrid_win_overall += 1

  print()

Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?
Độ toàn diện:
Câu trả lời 1
Sự đa dạng:
Câu trả lời 2
Độ hữu ích:
Câu trả lời 1
Kết quả cuối: 
Câu trả lời 1

Phân biệt tài sản chung và tài sản riêng của vợ chồng theo quy định pháp luật, và hậu quả pháp lý khi không phân định rõ ràng?
Độ toàn diện:
Câu trả lời 1
Sự đa dạng:
Câu trả lời 2
Độ hữu ích:
Câu trả lời 1
Kết quả cuối: 
Câu trả lời 1

Quyền yêu cầu ly hôn theo từng trường hợp (thuận tình, một bên yêu cầu) và nguyên tắc chia tài sản được áp dụng ra sao?
Độ toàn diện:
Câu trả lời 1
Sự đa dạng:
Câu trả lời 2
Độ hữu ích:
Câu trả lời 1
Kết quả cuối: 
Câu trả lời 1

Làm thế nào để tư vấn khách hàng về quyền và nghĩa vụ trong quan hệ cha mẹ và con, đặc biệt trong việc cấp dưỡng và xác định cha mẹ con?
Độ toàn diện:
Câu trả lời 1
Sự đa dạng:
Câu trả lời 2
Độ hữu ích:
Câu trả lời 1
Kết quả cuối: 
Câu trả lời 1

Những hành vi nào bị nghiêm cấm trong luật hôn nhân và 

## Kết quả lần 1 : hy->na

In [59]:
hybrid_win_overall

120

In [60]:
hybrid_win_do_huu_ich

120

In [61]:
hybrid_win_su_da_dang

60

In [62]:
hybrid_win_toan_dien

122

In [63]:
with open("RESULTS_GEN_QUESTIONS_Ver1.json", "w") as outfile:
    json.dump(RESULTS_GEN_QUESTIONS, outfile, indent=4)

### Nhận xét:
Lần 1: thứ Tự là Hy trước Na
--> Phần trăm thắng = 120/125 ~ 96%

## Kết quả lần 2

In [70]:
hybrid_win_toan_dien = 0
hybrid_win_su_da_dang = 0
hybrid_win_do_huu_ich = 0
hybrid_win_overall = 0

for res in RESULTS_GEN_QUESTIONS:
  print(res['q'])
  #print(res['res'])

  r = res['res']

  print('Độ toàn diện:')
  print(r['Độ toàn diện']['Người thắng'])
  win = r['Độ toàn diện']['Người thắng']
  if win == 'Câu trả lời 1':
    hybrid_win_toan_dien += 1

  print('Sự đa dạng:')
  print(r['Sự đa dạng']['Người thắng'])
  if r['Sự đa dạng']['Người thắng'] == 'Câu trả lời 1':
    hybrid_win_su_da_dang += 1

  print('Độ hữu ích:')
  print(r['Độ hữu ích']['Người thắng'])
  if r['Độ hữu ích']['Người thắng'] == 'Câu trả lời 1':
    hybrid_win_do_huu_ich += 1

  print('Kết quả cuối: ')
  print(r['Người thắng tổng thể']['Người thắng'])
  if r['Người thắng tổng thể']['Người thắng'] == 'Câu trả lời 1':
    hybrid_win_overall += 1

  print()

Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?
Độ toàn diện:
Câu trả lời 1
Sự đa dạng:
Câu trả lời 1
Độ hữu ích:
Câu trả lời 1
Kết quả cuối: 
Câu trả lời 1

Phân biệt tài sản chung và tài sản riêng của vợ chồng theo quy định pháp luật, và hậu quả pháp lý khi không phân định rõ ràng?
Độ toàn diện:
Câu trả lời 1
Sự đa dạng:
Câu trả lời 2
Độ hữu ích:
Câu trả lời 1
Kết quả cuối: 
Câu trả lời 1

Quyền yêu cầu ly hôn theo từng trường hợp (thuận tình, một bên yêu cầu) và nguyên tắc chia tài sản được áp dụng ra sao?
Độ toàn diện:
Câu trả lời 1
Sự đa dạng:
Câu trả lời 2
Độ hữu ích:
Câu trả lời 1
Kết quả cuối: 
Câu trả lời 1

Làm thế nào để tư vấn khách hàng về quyền và nghĩa vụ trong quan hệ cha mẹ và con, đặc biệt trong việc cấp dưỡng và xác định cha mẹ con?
Độ toàn diện:
Câu trả lời 1
Sự đa dạng:
Câu trả lời 1
Độ hữu ích:
Câu trả lời 1
Kết quả cuối: 
Câu trả lời 1

Những hành vi nào bị nghiêm cấm trong luật hôn nhân và 

In [71]:
hybrid_win_overall

101

In [72]:
hybrid_win_do_huu_ich

101

In [73]:
hybrid_win_su_da_dang

20

In [74]:
hybrid_win_toan_dien

106

In [75]:
with open("RESULTS_GEN_QUESTIONS_Ver2.json", "w") as outfile:
    json.dump(RESULTS_GEN_QUESTIONS, outfile, indent=4)

### Nhận xét
Lần 2 : Na -> Hy
Phần trắm thắng = 101/125 ~ 80,8%
==> phần trăm thắng của Hy = 19,2%

## Kết quả lần 3

In [83]:
RESULTS_GEN_QUESTIONS[2]

{'q': 'Quyền yêu cầu ly hôn theo từng trường hợp (thuận tình, một bên yêu cầu) và nguyên tắc chia tài sản được áp dụng ra sao?',
 'res': {'Độ toàn diện': {'Người thắng': 'Câu trả lời 1',
   'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn chi tiết hơn về quyền yêu cầu ly hôn và nguyên tắc chia tài sản, bao gồm các quy định cụ thể từ Luật Hôn nhân và Gia đình, cũng như các yếu tố cần xem xét trong từng trường hợp. Nó không chỉ nêu rõ các hình thức ly hôn mà còn đi sâu vào các nguyên tắc chia tài sản, đảm bảo rằng tất cả các khía cạnh của câu hỏi được bao quát.'},
  'Sự đa dạng': {'Người thắng': 'Câu trả lời 2',
   'Giải thích': 'Câu trả lời 2 trình bày các khía cạnh khác nhau của quyền yêu cầu ly hôn và nguyên tắc chia tài sản, bao gồm cả tài sản chung và tài sản riêng. Nó cũng đề cập đến sự can thiệp của Tòa án trong trường hợp không đạt được thỏa thuận, điều này mang lại một cái nhìn đa dạng hơn về vấn đề.'},
  'Độ hữu ích': {'Người thắng': 'Câu trả lời 1',
   'Giải thích': 'Câu trả

In [84]:
len(RESULTS_GEN_QUESTIONS)

250

In [86]:
RESULTS_GEN_QUESTIONS[125]

{'q': 'Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?',
 'res': {'Độ toàn diện': {'Người thắng': 'Câu trả lời 1',
   'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn chi tiết và đầy đủ về các điều kiện kết hôn theo quy định của pháp luật, bao gồm độ tuổi, sự tự nguyện, tình trạng hôn nhân, quan hệ huyết thống, quy trình đăng ký và giấy tờ cần thiết. Trong khi đó, Câu trả lời 2 thiếu một số khía cạnh như quan hệ huyết thống và giấy tờ cần thiết.'},
  'Sự đa dạng': {'Người thắng': 'Câu trả lời 2',
   'Giải thích': 'Câu trả lời 2 đề cập đến nhiều khía cạnh khác nhau của việc kết hôn, bao gồm năng lực hành vi dân sự và các trường hợp cấm kết hôn. Điều này mang lại nhiều góc nhìn hơn về các vấn đề pháp lý liên quan đến kết hôn, trong khi Câu trả lời 1 chủ yếu tập trung vào các điều kiện cơ bản.'},
  'Độ hữu ích': {'Người thắng': 'Câu trả lời 1',
   'Giải thích': 'Câu trả lời 1 không chỉ cung cấp thông tin chi tiết m

In [87]:
VER3 = RESULTS_GEN_QUESTIONS[125:]

In [88]:
len(VER3)

125

In [90]:
VER3[-1]

{'q': 'Các phương án tư vấn nào phù hợp với văn hóa xã hội và pháp luật để giúp các gia đình xử lý các mâu thuẫn gia đình mà không làm tổn thương các thành viên?',
 'res': {'Độ toàn diện': {'Người thắng': 'Câu trả lời 1',
   'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn chi tiết và toàn diện về các phương án tư vấn, bao gồm hòa giải cấp cơ sở, sự tham gia của người có uy tín, tư vấn pháp lý và giáo dục. Mỗi phương án đều được giải thích rõ ràng với lợi ích cụ thể, giúp người đọc hiểu rõ hơn về cách thức áp dụng trong thực tế.'},
  'Sự đa dạng': {'Người thắng': 'Câu trả lời 2',
   'Giải thích': 'Câu trả lời 2 mang lại sự đa dạng hơn trong các phương án tư vấn, bao gồm hòa giải, tư vấn tâm lý và tư vấn pháp luật. Nó không chỉ đề cập đến hòa giải mà còn mở rộng ra các khía cạnh tâm lý và pháp lý, giúp người đọc có cái nhìn toàn diện hơn về các lựa chọn có sẵn.'},
  'Độ hữu ích': {'Người thắng': 'Câu trả lời 1',
   'Giải thích': 'Câu trả lời 1 cung cấp thông tin cụ thể và rõ ràng về c

In [96]:
VER3[0]['res']

{'Độ toàn diện': {'Người thắng': 'Câu trả lời 1',
  'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn chi tiết và đầy đủ về các điều kiện kết hôn theo quy định của pháp luật, bao gồm độ tuổi, sự tự nguyện, tình trạng hôn nhân, quan hệ huyết thống, quy trình đăng ký và giấy tờ cần thiết. Trong khi đó, Câu trả lời 2 thiếu một số khía cạnh như quan hệ huyết thống và giấy tờ cần thiết.'},
 'Sự đa dạng': {'Người thắng': 'Câu trả lời 2',
  'Giải thích': 'Câu trả lời 2 đề cập đến nhiều khía cạnh khác nhau của việc kết hôn, bao gồm năng lực hành vi dân sự và các trường hợp cấm kết hôn. Điều này mang lại nhiều góc nhìn hơn về các vấn đề pháp lý liên quan đến kết hôn, trong khi Câu trả lời 1 chủ yếu tập trung vào các điều kiện cơ bản.'},
 'Độ hữu ích': {'Người thắng': 'Câu trả lời 1',
  'Giải thích': 'Câu trả lời 1 không chỉ cung cấp thông tin chi tiết mà còn kết luận rõ ràng về tầm quan trọng của việc tuân thủ các quy định pháp luật, giúp người đọc hiểu rõ hơn về vấn đề và đưa ra quyết định sá

In [100]:
num_overall = 0
num_td = 0
num_dd = 0
num_hi = 0

for res in VER3:
  #print(i)
  print(res['num'])

  if res['num'] % 2 == 0:
    # ctl 1 = na -> ctl2 = hy
    i = res['res']
    if i['Độ toàn diện']['Người thắng'] == 'Câu trả lời 2':
      num_td += 1
    if i['Sự đa dạng']['Người thắng'] == 'Câu trả lời 2':
      num_dd += 1
    if i['Độ hữu ích']['Người thắng'] == 'Câu trả lời 2':
      num_hi += 1
    if i['Người thắng tổng thể']['Người thắng'] == 'Câu trả lời 2':
      num_overall += 1

  else:
    # ctl1 = hy
    # ctl 1 = na -> ctl2 = hy
    i = res['res']
    if i['Độ toàn diện']['Người thắng'] == 'Câu trả lời 1':
      num_td += 1
    if i['Sự đa dạng']['Người thắng'] == 'Câu trả lời 1':
      num_dd += 1
    if i['Độ hữu ích']['Người thắng'] == 'Câu trả lời 1':
      num_hi += 1
    if i['Người thắng tổng thể']['Người thắng'] == 'Câu trả lời 1':
      num_overall += 1


3
4
4
1
5
6
5
15
15
10
11
4
10
15
8
4
9
7
17
13
5
8
13
19
9
16
13
10
5
16
19
2
14
9
1
9
16
15
7
2
10
1
7
17
14
2
9
3
3
18
6
7
18
2
4
5
15
7
2
9
7
17
11
15
12
17
14
14
10
4
8
7
14
7
11
3
15
2
2
11
1
3
13
13
14
6
5
17
14
17
14
18
12
3
7
6
5
15
1
12
2
1
14
3
16
9
18
10
15
1
2
14
17
3
7
5
13
13
4
15
12
3
14
16
2


In [101]:
num_overall

80

In [102]:
num_td

81

In [103]:
num_dd

73

In [104]:
num_hi

80

In [105]:
with open("RESULTS_GEN_QUESTIONS_Ver3.json", "w", encoding= 'utf-8') as outfile:
    json.dump(RESULTS_GEN_QUESTIONS, outfile, indent=4)

## Lần 3
--> Hy thắng 64%

# Kết quả lần 4

In [111]:
len(VER4)

125

In [112]:
num_overall = 0
num_td = 0
num_dd = 0
num_hi = 0

for res in VER4:
  #print(i)
  print(res['num'])

  if res['num'] % 2 == 0:
    # ctl 1 = na -> ctl2 = hy
    i = res['res']
    if i['Độ toàn diện']['Người thắng'] == 'Câu trả lời 2':
      num_td += 1
    if i['Sự đa dạng']['Người thắng'] == 'Câu trả lời 2':
      num_dd += 1
    if i['Độ hữu ích']['Người thắng'] == 'Câu trả lời 2':
      num_hi += 1
    if i['Người thắng tổng thể']['Người thắng'] == 'Câu trả lời 2':
      num_overall += 1

  else:
    # ctl1 = hy
    # ctl 1 = na -> ctl2 = hy
    i = res['res']
    if i['Độ toàn diện']['Người thắng'] == 'Câu trả lời 1':
      num_td += 1
    if i['Sự đa dạng']['Người thắng'] == 'Câu trả lời 1':
      num_dd += 1
    if i['Độ hữu ích']['Người thắng'] == 'Câu trả lời 1':
      num_hi += 1
    if i['Người thắng tổng thể']['Người thắng'] == 'Câu trả lời 1':
      num_overall += 1

39
24
10
39
15
42
34
42
23
5
31
2
8
16
41
38
25
39
20
17
41
5
10
7
8
36
3
38
14
20
6
29
20
45
1
29
38
38
26
7
17
21
30
44
14
6
12
30
39
16
3
7
27
27
38
36
21
33
30
29
22
4
14
23
25
37
30
35
23
41
45
40
9
10
11
44
35
15
16
21
17
19
7
12
12
22
19
22
44
24
37
1
34
6
7
7
20
16
26
15
38
28
14
4
1
4
17
18
19
8
29
44
9
6
27
22
8
32
42
39
16
17
18
43
5


In [114]:
num_overall

71

In [115]:
num_td

69

In [116]:
num_dd

82

In [117]:
num_hi

71

In [136]:
with open("RESULTS_GEN_QUESTIONS_Ver4.json", "w", encoding= 'utf-8') as outfile:
    json.dump(VER4, outfile, indent=4)

### Hy thắng : 71/125 ~ 56,8%

## Kiểm tra lần 5, sử dụng Chat-gpt-4o

In [127]:
VER5 = []

In [128]:
VER5

[]

In [129]:
for i in tqdm(range(len(LIGHTRAG_HYBRID_QA))):
  hybrid = LIGHTRAG_HYBRID_QA[i]
  naive = LIGHTRAG_NAIVE_QA[i]

  print(LIGHTRAG_HYBRID_QA[i]['q'])
  print(LIGHTRAG_NAIVE_QA[i]['q'])

  prompt = PROMPT['evaluation_gen_questions']

  prompt = prompt.replace('<<q>>', LIGHTRAG_HYBRID_QA[i]['q'])

  number = random.randint(1, 45)
  print(number)
  if number % 2 == 0:
    prompt = prompt.replace('<<a1>>', LIGHTRAG_NAIVE_QA[i]['a']) # hy # na
    prompt = prompt.replace('<<a2>>', LIGHTRAG_HYBRID_QA[i]['a']) #na # hy
  else:
    prompt = prompt.replace('<<a1>>', LIGHTRAG_HYBRID_QA[i]['a']) # hy # na
    prompt = prompt.replace('<<a2>>', LIGHTRAG_NAIVE_QA[i]['a'])

  res = get_valid_json_gpt40mini(prompt)

  if res == "Not":
    continue

  a= {}
  a['q'] = LIGHTRAG_HYBRID_QA[i]['q']
  a['res'] = res
  a['num'] = number

  print(a['res'])

  VER5.append(a)

  0%|          | 0/125 [00:00<?, ?it/s]

Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?
Các điều kiện kết hôn được quy định trong luật có những điểm nào cần lưu ý để tư vấn khách hàng tránh vi phạm pháp luật?
9
attempt : 0


  1%|          | 1/125 [00:09<18:47,  9.10s/it]

gpt-4o
Định dạng JSON hợp lệ!
{'Độ toàn diện': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp một danh sách chi tiết và đầy đủ về các điều kiện kết hôn theo Luật Hôn nhân và Gia đình 2014, bao gồm độ tuổi, tự nguyện, tình trạng hôn nhân, quan hệ huyết thống, quy trình đăng ký và giấy tờ cần thiết. Câu trả lời 2 mặc dù cũng đề cập đến nhiều điểm quan trọng nhưng thiếu một số chi tiết như quan hệ huyết thống và giấy tờ cần thiết.'}, 'Sự đa dạng': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 đưa ra nhiều góc nhìn khác nhau về các điều kiện kết hôn, bao gồm cả năng lực hành vi dân sự và các trường hợp cấm kết hôn như kết hôn giả tạo, tảo hôn, và lừa dối. Điều này mang lại một cái nhìn đa chiều hơn về các vấn đề pháp lý liên quan đến kết hôn.'}, 'Độ hữu ích': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 có tính hữu ích cao hơn vì nó cung cấp một hướng dẫn rõ ràng và chi tiết về từng điều kiện kết hôn, giúp người đọc dễ dàng nắm bắt v

  2%|▏         | 2/125 [00:14<14:43,  7.19s/it]

...

gpt-4o
Định dạng JSON hợp lệ!
{'Độ toàn diện': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 cung cấp một cái nhìn toàn diện hơn về các nguyên tắc pháp lý liên quan đến phân chia tài sản khi ly hôn. Nó không chỉ đề cập đến các nguyên tắc cơ bản như phân loại tài sản và thỏa thuận tài sản mà còn mở rộng sang các yếu tố như quyền lợi và nghĩa vụ tài sản, vai trò của tòa án, và yếu tố lỗi trong phân chia tài sản. Điều này cho thấy một sự bao quát rộng hơn về các khía cạnh pháp lý cần xem xét.'}, 'Sự đa dạng': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 thể hiện sự đa dạng hơn trong việc đề cập đến nhiều khía cạnh khác nhau của vấn đề, bao gồm cả yếu tố lỗi và quyền lợi tài sản gắn liền với nhân thân. Nó cung cấp nhiều góc nhìn khác nhau về cách thức phân chia tài sản, từ thỏa thuận tự nguyện đến sự can thiệp của tòa án, điều này giúp người đọc có cái nhìn đa chiều hơn về vấn đề.'}, 'Độ hữu ích': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 

 99%|█████████▉| 124/125 [13:53<00:07,  7.93s/it]

gpt-4o
Định dạng JSON hợp lệ!
{'Độ toàn diện': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 cung cấp một cái nhìn chi tiết hơn về các quy định pháp lý liên quan đến quyền lợi của trẻ em trong trường hợp ly hôn, bao gồm cả việc phân chia tài sản, nghĩa vụ cấp dưỡng, và quyền thăm nom. Ngoài ra, nó cũng đề cập đến vai trò của Tòa án và tư vấn sức khỏe tâm lý cho trẻ, điều này làm cho câu trả lời này toàn diện hơn.'}, 'Sự đa dạng': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 đưa ra nhiều góc nhìn khác nhau về việc bảo vệ quyền lợi của trẻ em, từ khía cạnh pháp lý đến tâm lý. Nó không chỉ tập trung vào các quy định pháp luật mà còn đề cập đến các biện pháp tư vấn tâm lý, điều này mang lại một cái nhìn đa chiều hơn về vấn đề.'}, 'Độ hữu ích': {'Người thắng': 'Câu trả lời 2', 'Giải thích': 'Câu trả lời 2 hữu ích hơn vì nó cung cấp thông tin chi tiết và rõ ràng về các quy định pháp lý và các bước phụ huynh có thể thực hiện để bảo vệ quyền lợi của trẻ. Nó cũn

100%|██████████| 125/125 [13:59<00:00,  6.71s/it]

gpt-4o
Định dạng JSON hợp lệ!
{'Độ toàn diện': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 cung cấp một cái nhìn toàn diện hơn về các phương án tư vấn phù hợp với văn hóa xã hội và pháp luật. Nó không chỉ liệt kê các phương án mà còn giải thích chi tiết về lợi ích của từng phương án, từ hòa giải cấp cơ sở, sự tham gia của người có uy tín, tư vấn pháp lý, đến giáo dục và tuyên truyền. Câu trả lời 2 mặc dù cũng đề cập đến các phương án tương tự nhưng thiếu sự chi tiết và không đi sâu vào lợi ích cụ thể của từng phương án.'}, 'Sự đa dạng': {'Người thắng': 'Câu trả lời 1', 'Giải thích': 'Câu trả lời 1 thể hiện sự đa dạng hơn bằng cách kết hợp các phương pháp truyền thống và hiện đại, bao gồm cả sự tham gia của người có uy tín và chức sắc tôn giáo, điều mà câu trả lời 2 không đề cập đến. Câu trả lời 2 chủ yếu tập trung vào hòa giải, tư vấn tâm lý và pháp luật mà không mở rộng ra các khía cạnh văn hóa và giáo dục như câu trả lời 1.'}, 'Độ hữu ích': {'Người thắng': 'Câu trả 

In [130]:
len(VER5)

125

In [144]:
num_overall = 0
num_td = 0
num_dd = 0
num_hi = 0

num_chan = 0

for res in VER5:
  #print(i)
  print(res['num'])

  if res['num'] % 2 == 0:
    num_chan+=1
    # ctl 1 = na -> ctl2 = hy
    i = res['res']
    if i['Độ toàn diện']['Người thắng'] == 'Câu trả lời 2':
      num_td += 1
    if i['Sự đa dạng']['Người thắng'] == 'Câu trả lời 2':
      num_dd += 1
    if i['Độ hữu ích']['Người thắng'] == 'Câu trả lời 2':
      num_hi += 1
    if i['Người thắng tổng thể']['Người thắng'] == 'Câu trả lời 2':
      num_overall += 1

  else:
    # ctl1 = hy
    # ctl 1 = na -> ctl2 = hy
    i = res['res']
    if i['Độ toàn diện']['Người thắng'] == 'Câu trả lời 1':
      num_td += 1
    if i['Sự đa dạng']['Người thắng'] == 'Câu trả lời 1':
      num_dd += 1
    if i['Độ hữu ích']['Người thắng'] == 'Câu trả lời 1':
      num_hi += 1
    if i['Người thắng tổng thể']['Người thắng'] == 'Câu trả lời 1':
      num_overall += 1

9
45
8
42
7
25
13
9
31
4
18
23
17
44
29
27
14
30
27
44
2
6
6
2
30
14
3
4
22
42
22
3
29
6
8
37
2
20
14
26
21
3
11
7
2
10
16
40
3
41
15
24
30
23
24
13
23
39
18
32
34
22
2
7
13
27
3
16
10
30
45
37
43
4
5
38
33
5
9
3
18
19
9
26
32
30
18
38
4
16
2
30
13
17
19
4
10
18
7
31
41
23
10
44
36
29
44
7
7
3
16
33
21
43
22
6
36
27
33
42
11
8
6
12
44


In [145]:
num_chan

67

In [139]:
num_overall

69

In [140]:
num_td

67

In [141]:
num_dd

72

In [142]:
num_hi

70

In [137]:
with open("RESULTS_GEN_QUESTIONS_Ver5.json", "w", encoding= 'utf-8') as outfile:
    json.dump(VER5, outfile, indent=4)

### Hy thắng : 55,2 %

# Kết luận

96+19,2+64+56,8+55,2

~ 58,24%